# CSE555 – Deep Learning (Fall 2023)
# Homework_1

## Part 2 : 2D Object Recognition using CNNs

Data Set :  https://github.com/TimoFlesch/2D-Shape-Generator

In [ ]:
Requirements:
pip install python==3.6.1
pip install matplotlib==2.0.2
pip install numpy==1.12.1
pip install scipy==0.19.0
pip install pycairo==1.13.3

In [ ]:
#!pip install cairo
!pip3 install pycairo


In [ ]:
pip install scipy

# Import libraries & data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
import itertools
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [ ]:
"""
function to generate 2D shape stimulus sets, heavily inspired by Higgins et al 2016

Each image consists of a geometric shape (polygon, ellipse, star) on a (default: black) background. 
Individual shape instances can be modulated parametrically in terms of
- translation (x,y)
- rotation
- scale 
- colour

Timo Flesch, 2017
"""
import cairo 
import argparse
import numpy as np
from paint_studio.painter import drawStimuli
from fio.saver            import saveData


# PARAMETERS ----------------------------------------------------------------------------
parser = argparse.ArgumentParser()
# general -------------------------------------------------------------------------------
parser.add_argument('--outdir','-o',           type=str,   nargs='+', default='./output/', 
	                                           help='Output directory (default: ./output/')
parser.add_argument('--name',                  type=str,   nargs='+', default=['stim2D_'],
                                               help='File Name (default: stim2D_')
parser.add_argument('--parallel',              type=int,   nargs=1,   default=0, 
	                                           help='Do Parallel Processing (default: 0)')

# canvas --------------------------------------------------------------------------------
parser.add_argument('--canvas_size',           type=int,   nargs='+', default=[128, 128], # change   pixel 
                                               help='Size of Image Canvas (default: 128x128px)') 
parser.add_argument('--canvas_bgcol',          type=float, nargs='+', default=[0.,0.,0.], 
	                                           help='Background Colour of Image Canvas (RGB,default: 0 0 0)')

# stimuli -------------------------------------------------------------------------------
parser.add_argument('--num_stimuli',           type=int,   nargs=1,   default=1, 
	                                           help='Number of stimuli per shape to generate (default: 1)') 
parser.add_argument('--num_transformations',   type=int,              default=0, 
	                                           help='Number of linearly spaced transformations (default:0)')
parser.add_argument('--shapes',                type=str,   nargs='+', default=['rect','ellipse', 'oval', 'poly6', 'poly7', 'star5', 'star8'], # those  all shape names  data set
	                                           help='Shapes (rect, polyNUM,starNUM, ellipse) ')
parser.add_argument('--to_transform',          type=str,   nargs='+', default=[],
	                                           help='dimensions to transform with num_transformations. \
	                                           All other dims kept constant (scale,rota,trx,try,colour; default: None)')
parser.add_argument('--stim_poly_size',        type=int,                default=20,
	                                           help='Polygon "Radius"  (default: 50)')
parser.add_argument('--stim_star_size',        type=float,   nargs='+', default=[20,10],
	                                           help='Star Radii (outer, inner; default: [50,25])')
parser.add_argument('--stim_ellipse_ratio',    type=float, nargs='+', default=[1,.5],
	                                           help='Scaling of Circle to generate Ellipse')
parser.add_argument('--stim_rect_ratio',    type=float, nargs='+', default=[.5,1],
	                                           help='Scaling of square to generate rect')
parser.add_argument('--stim_scale',            type=float, nargs='+', default=[1,1],
	                                           help='Stimulus Scaling (default: (1,1)')
parser.add_argument('--stim_rota',             type=float,            default=0,
                                               help='Stimulus Rotation (deg; default: 0)')
parser.add_argument('--stim_trx',              type=float,             default=125,
	                                           help='Stimulus Translation along x (default: centre)')
parser.add_argument('--stim_try',              type=float,             default=125,
	                                           help='Stimulus Translation along y (default: 0)')
parser.add_argument('--stim_colour',           type=float,  nargs='+', default=[1., 1., 1.],
	                                           help='Stimulus Colour (default: [1.,1.,1.]')

# transformation ranges -----------------------------------------------------------------
parser.add_argument('--rng_ratio',             type=float, nargs='+', default=[.2, 5.],
                                               help='Shape width to height ratio (min,max; defaults:[.2, 5.]')
parser.add_argument('--rng_trx',               type=float, nargs='+', default=[.10, .90],
                                               help='Translation along x in decimal fractions (min,max; defaults:[.05, .95]')
parser.add_argument('--rng_try',               type=float, nargs='+', default=[.10, .90], 
	                                           help='Translation along y in decimal fractions (min,max; defaults:[.05, .95]')
parser.add_argument('--rng_rota',              type=int,   nargs='+', default=[-45, 45], 
	                                           help='Rotation in deg (min,max; defaults:[-45, 45]')
parser.add_argument('--rng_scale',             type=float, nargs='+', default=[.5, 1.5],
                                               help='Scale decimal fractions (min,max; defaults:[.2, 1.]')


FLAGS,_ = parser.parse_known_args() # ignore unspecified args
FLAGS.rng_trx = np.multiply(FLAGS.rng_trx,FLAGS.canvas_size)
FLAGS.rng_try = np.multiply(FLAGS.rng_try,FLAGS.canvas_size)
print(FLAGS)

def main(argv=None):	
	all_IMGs,all_idces = drawStimuli(FLAGS)
	data = {}
	data['images'] = all_IMGs # store all the images
	data['params'] = vars(FLAGS) # store all the parameters
	data['idces']  = all_idces  # for each image, store the feature values. (feature correspondence encoded in FLAGS.to_transform (same order))
	saveData(data,FLAGS.to_transform,FLAGS.shapes,FLAGS.outdir)
if __name__ == '__main__':
	""" start the fun"""
	main()


# this code was executed but could not generate the required images. Therefore only the model was installed.  If I could generate images I could try it on the model.

# Loading Images:

In [ ]:
import cv2
import numpy as np

# Image paths
image_paths = ["path/to/image1.jpg", "path/to/image2.jpg", ...]

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224, 1))
    image = image / 255.0  # Scaling pixel values to [0, 1]
    return image

# Load all images in an array
images = [load_and_preprocess_image(image_path) for image_path in image_paths]


# Creating Labels:

In [ ]:
# For example, match each shape  image to a class number
labels = [0, 1, 2, 3, 4, 5, 6, 7]

In [ ]:
# Convert images to a NumPy array
X = np.array(images)

# Splitting into Training and Test Sets:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()

# AlexNet architecture
model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
model.add(layers.Conv2D(256, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
model.add(layers.Conv2D(384, (3, 3), activation='relu'))
model.add(layers.Conv2D(384, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))  # Class number 8

# Model compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Model training

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
 # Model Performance

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')
